In [4]:
!pip install -q transformers

In [5]:
import pandas as pd
import numpy as np
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, RandomSampler
from torch.nn import functional as F
from sklearn.model_selection import train_test_split

In [6]:
df=pd.read_csv("/content/sangeetha.csv")

In [7]:
df.head()

,NBa7we src,d4r55,RfnDt,RfnDt 2,eaLgGf src,hCCjke src,hCCjke src 2,hCCjke src 3,hCCjke src 4,hCCjke src 5,rsqaWe,wiI7pd,w8nwRe,dSlJg src,znYl0,dSlJg src 2,znYl0 2,RfDO5c
0,https://lh3.googleusercontent.com/a-/AD_cMMRJK...,Subaash Viswanathan,· 281 reviews,Local Guide,https://www.gstatic.com/images/icons/material/...,https://maps.gstatic.com/consumer/images/icons...,https://maps.gstatic.com/consumer/images/icons...,https://maps.gstatic.com/consumer/images/icons...,https://maps.gstatic.com/consumer/images/icons...,https://maps.gstatic.com/consumer/images/icons...,a day ago,I went for breakfast on a Saturday. The restau...,More,https://www.gstatic.com/images/icons/material/...,Like,https://www.gstatic.com/images/icons/material/...,Share,NaN
1,https://lh3.googleusercontent.com/a-/AD_cMMRno...,Maathanghi Vijayaragavan,· 24 reviews,Local Guide,https://www.gstatic.com/images/icons/material/...,https://maps.gstatic.com/consumer/images/icons...,https://maps.gstatic.com/consumer/images/icons...,https://maps.gstatic.com/consumer/images/icons...,https://maps.gstatic.com/consumer/images/icons...,https://maps.gstatic.com/consumer/images/icons...,a week ago,Sangeetha Hotel has been here for a very long ...,More,https://www.gstatic.com/images/icons/material/...,Like,https://www.gstatic.com/images/icons/material/...,Share,NaN
2,https://lh3.googleusercontent.com/a-/AD_cMMStp...,Rajiv Gopinath,· 28 reviews,Local Guide,https://www.gstatic.com/images/icons/material/...,https://maps.gstatic.com/consumer/images/icons...,https://maps.gstatic.com/consumer/images/icons...,https://maps.gstatic.com/consumer/images/icons...,https://maps.gstatic.com/consumer/images/icons...,https://maps.gstatic.com/consumer/images/icons...,3 months ago,"Had people who were raving about Sangeetha, so...",More,https://www.gstatic.com/images/icons/material/...,Like,https://www.gstatic.com/images/icons/material/...,Share,NaN
3,https://lh3.googleusercontent.com/a-/AD_cMMQz0...,Kamala Palaniappan,· 27 reviews,Local Guide,https://www.gstatic.com/images/icons/material/...,https://maps.gstatic.com/consumer/images/icons...,https://maps.gstatic.com/consumer/images/icons...,https://maps.gstatic.com/consumer/images/icons...,https://maps.gstatic.com/consumer/images/icons...,https://maps.gstatic.com/consumer/images/icons...,3 months ago,The best for south Indian vegetarian foodies. ...,More,https://www.gstatic.com/images/icons/material/...,Like,https://www.gstatic.com/images/icons/material/...,Share,NaN
4,https://lh3.googleusercontent.com/a-/AD_cMMQbm...,David Kwok,· 91 reviews,Local Guide,https://www.gstatic.com/images/icons/material/...,https://maps.gstatic.com/consumer/images/icons...,https://maps.gstatic.com/consumer/images/icons...,https://maps.gstatic.com/consumer/images/icons...,https://maps.gstatic.com/consumer/images/icons...,https://maps.gstatic.com/consumer/images/icons...,5 months ago,Sangeetha is a chain of restaurants in Chennai...,More,https://www.gstatic.com/images/icons/material/...,Like,https://www.gstatic.com/images/icons/material/...,Share,NaN


In [8]:
df = df[['hCCjke src', 'hCCjke src 2', 'hCCjke src 3','hCCjke src 4','hCCjke src 5','wiI7pd']]

In [9]:
new_column_names = {
    'hCCjke src': '1 star',
    'hCCjke src 2': '2 star',
    'hCCjke src 3': '3 star',
    'hCCjke src 4': '4 star',
    'hCCjke src 5': '5 star',
    'wiI7pd': 'text'
}

df = df.rename(columns=new_column_names)

In [10]:
df=df.dropna()

In [11]:
df = df.replace({
    'https://maps.gstatic.com/consumer/images/icons/2x/ic_star_rate_empty_14.png': 0,
    'https://maps.gstatic.com/consumer/images/icons/2x/ic_star_rate_14.png': 1
})

In [12]:

df['star rating'] = df[['1 star', '2 star', '3 star', '4 star', '5 star']].sum(axis=1)


In [13]:
df=df[['text','star rating']]

In [14]:
df = df[df['star rating'] != 3]

In [15]:
df['sentiment'] = df['star rating'].apply(lambda x: 'negative' if x in [1, 2] else 'positive')

In [16]:
df=df[['text','sentiment']]

In [17]:

# Load pre-trained BERT model and tokenizer
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)

# Set device (CPU or GPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Example DataFrame with sentiment data


# Map sentiment labels to integers
label_map = {'positive': 1, 'negative': 0}
df['label'] = df['sentiment'].map(label_map)

# Preprocess the data
texts = df['text'].values
labels = df['label'].values

# Split the data into training and evaluation sets
train_texts, eval_texts, train_labels, eval_labels = train_test_split(texts, labels, test_size=0.2, random_state=42)

# Tokenize and convert the text samples to input tensors
train_inputs = tokenizer(train_texts.tolist(), padding=True, truncation=True, return_tensors='pt')
eval_inputs = tokenizer(eval_texts.tolist(), padding=True, truncation=True, return_tensors='pt')

train_input_ids = train_inputs['input_ids'].to(device)
train_attention_mask = train_inputs['attention_mask'].to(device)
train_labels_tensor = torch.tensor(train_labels, dtype=torch.long).to(device)

eval_input_ids = eval_inputs['input_ids'].to(device)
eval_attention_mask = eval_inputs['attention_mask'].to(device)
eval_labels_tensor = torch.tensor(eval_labels, dtype=torch.long).to(device)

# Create data loaders
batch_size = 4
train_data = torch.utils.data.TensorDataset(train_input_ids, train_attention_mask, train_labels_tensor)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

eval_data = torch.utils.data.TensorDataset(eval_input_ids, eval_attention_mask, eval_labels_tensor)
eval_dataloader = DataLoader(eval_data, batch_size=batch_size)

# Set training parameters
num_epochs = 5
learning_rate = 2e-5

# Optimizer and loss function
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
loss_fn = torch.nn.CrossEntropyLoss()

# Fine-tuning loop
model.train()
for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}")
    print("-------------------------")
    total_loss = 0

    for step, batch in enumerate(train_dataloader):
        input_ids, attention_mask, labels = batch

        optimizer.zero_grad()

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        total_loss += loss.item()

        loss.backward()
        optimizer.step()

    avg_loss = total_loss / len(train_dataloader)
    print(f"Training loss: {avg_loss}")

    # Evaluation
    model.eval()
    eval_loss = 0
    num_correct = 0
    num_total = 0

    with torch.no_grad():
        for batch in eval_dataloader:
            input_ids, attention_mask, labels = batch

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss

            eval_loss += loss.item()

            logits = outputs.logits
            probabilities = F.softmax(logits, dim=1)
            predicted_labels = torch.argmax(probabilities, dim=1)

            num_correct += torch.sum(predicted_labels == labels).item()
            num_total += len(labels)

    eval_accuracy = num_correct / num_total
    avg_eval_loss = eval_loss / len(eval_dataloader)
    print(f"Validation loss: {avg_eval_loss}")
    print(f"Validation accuracy: {eval_accuracy}")

# Save the fine-tuned model
model.save_pretrained("fine_tuned_bert_model")
tokenizer.save_pretrained("fine_tuned_bert_model")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/5
-------------------------
Training loss: 0.2932548893361606
Validation loss: 0.2826920057861851
Validation accuracy: 0.8709677419354839
Epoch 2/5
-------------------------
Training loss: 0.1163805864642483
Validation loss: 0.10563574943882532
Validation accuracy: 0.9612903225806452
Epoch 3/5
-------------------------
Training loss: 0.037488510420201224
Validation loss: 0.17642796258382404
Validation accuracy: 0.9419354838709677
Epoch 4/5
-------------------------
Training loss: 0.005678121568997959
Validation loss: 0.09072657247545579
Validation accuracy: 0.967741935483871
Epoch 5/5
-------------------------
Training loss: 0.0009194829298240773
Validation loss: 0.09926142284828716
Validation accuracy: 0.9612903225806452


('fine_tuned_bert_model/tokenizer_config.json',
 'fine_tuned_bert_model/special_tokens_map.json',
 'fine_tuned_bert_model/vocab.txt',
 'fine_tuned_bert_model/added_tokens.json')

In [18]:
df1=pd.read_csv('/content/rayars_mess.csv')

In [19]:
df1 = df1[['hCCjke src', 'hCCjke src 2', 'hCCjke src 3','hCCjke src 4','hCCjke src 5','wiI7pd']]

In [20]:
df1 = df1.rename(columns=new_column_names)


In [21]:
df1=df1.dropna()

In [22]:
df1 = df1.replace({
    'https://maps.gstatic.com/consumer/images/icons/2x/ic_star_rate_empty_14.png': 0,
    'https://maps.gstatic.com/consumer/images/icons/2x/ic_star_rate_14.png': 1
})

In [23]:
df1['star rating'] = df1[['1 star', '2 star', '3 star', '4 star', '5 star']].sum(axis=1)

In [24]:
df1=df1[['text','star rating']]

In [26]:
df1 = df1[df1['star rating'] != 3]

In [27]:
df1['sentiment'] = df1['star rating'].apply(lambda x: 'negative' if x in [1, 2] else 'positive')

<ipython-input-27-f8439469f2b2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['sentiment'] = df1['star rating'].apply(lambda x: 'negative' if x in [1, 2] else 'positive')


In [28]:
df1=df1[['text','sentiment']]

In [29]:

# Load the fine-tuned BERT model and tokenizer
model_dir = "fine_tuned_bert_model"
model = BertForSequenceClassification.from_pretrained(model_dir)
tokenizer = BertTokenizer.from_pretrained(model_dir)

# Set device (CPU or GPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Example DataFrame with the new dataset
new_df = df1

# Map sentiment labels to integers
label_map = {'positive': 1, 'negative': 0}
new_df['label'] = new_df['sentiment'].map(label_map)

# Preprocess the new dataset
new_texts = new_df['text'].values
new_labels = new_df['label'].values

# Rest of the code for preprocessing and inference remains the same

# Preprocess the new dataset


new_inputs = tokenizer(new_texts.tolist(), padding=True, truncation=True, return_tensors='pt')
new_input_ids = new_inputs['input_ids'].to(device)
new_attention_mask = new_inputs['attention_mask'].to(device)
new_labels_tensor = torch.tensor(new_labels, dtype=torch.long).to(device)

new_data = torch.utils.data.TensorDataset(new_input_ids, new_attention_mask, new_labels_tensor)
new_dataloader = DataLoader(new_data, batch_size=4)

# Set the model to evaluation mode
model.eval()

# Perform inference on the new dataset
predictions = []
with torch.no_grad():
    for batch in new_dataloader:
        input_ids, attention_mask, labels = batch

        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        probabilities = F.softmax(logits, dim=1)
        predicted_labels = torch.argmax(probabilities, dim=1)

        predictions.extend(predicted_labels.cpu().tolist())

# Map the predicted labels to sentiment classes
sentiment_classes = ['negative', 'positive']
predicted_sentiments = [sentiment_classes[label] for label in predictions]

# Add predicted sentiments to the DataFrame
new_df['predicted_sentiment'] = predicted_sentiments

# Print the new DataFrame with predicted sentiments
print(new_df)


                                                  text sentiment  label  \
1    The food was tasty here. We tried idly, ghee p...  positive      1   
2    A highly hyped place in the social media arena...  positive      1   
3    Excellent breakfast spot. Need to be there ear...  positive      1   
4    Awesome food. Go to the beach for sunrise and ...  positive      1   
5    Few places are this good. I come here without ...  positive      1   
..                                                 ...       ...    ...   
789                      Excellent pongal vada tasty …  positive      1   
791  Mylapore Kacheri Road Market Beautiful homemad...  positive      1   
821                                              Uhtso  positive      1   
835                                        Great tease  positive      1   
885                                     V. nice bond …  positive      1   

    predicted_sentiment  
1              positive  
2              positive  
3              positi

In [30]:
from sklearn.metrics import accuracy_score

# Calculate accuracy score
true_labels = new_df['label'].values
predicted_labels = predictions

accuracy = accuracy_score(true_labels, predicted_labels)
print(f"Accuracy: {accuracy}")

Accuracy: 0.9434782608695652


In [31]:
df1=pd.read_csv('/content/rayars_mess.csv')
df1 = df1[['hCCjke src', 'hCCjke src 2', 'hCCjke src 3','hCCjke src 4','hCCjke src 5','wiI7pd']]
df1 = df1.rename(columns=new_column_names)
df1=df1.dropna()
df1 = df1.replace({
    'https://maps.gstatic.com/consumer/images/icons/2x/ic_star_rate_empty_14.png': 0,
    'https://maps.gstatic.com/consumer/images/icons/2x/ic_star_rate_14.png': 1
})
df1['star rating'] = df1[['1 star', '2 star', '3 star', '4 star', '5 star']].sum(axis=1)
df1=df1[['text','star rating']]
df1 = df1[df1['star rating'] != 3]
df1['sentiment'] = df1['star rating'].apply(lambda x: 'negative' if x in [1, 2] else 'positive')
df1=df1[['text','sentiment']]
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader
from torch.nn import functional as F

# Load the fine-tuned BERT model and tokenizer
model_dir = "fine_tuned_bert_model"
model = BertForSequenceClassification.from_pretrained(model_dir)
tokenizer = BertTokenizer.from_pretrained(model_dir)

# Set device (CPU or GPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Example DataFrame with the new dataset
new_df = df1

# Map sentiment labels to integers
label_map = {'positive': 1, 'negative': 0}
new_df['label'] = new_df['sentiment'].map(label_map)

# Preprocess the new dataset
new_texts = new_df['text'].values
new_labels = new_df['label'].values

# Rest of the code for preprocessing and inference remains the same

# Preprocess the new dataset


new_inputs = tokenizer(new_texts.tolist(), padding=True, truncation=True, return_tensors='pt')
new_input_ids = new_inputs['input_ids'].to(device)
new_attention_mask = new_inputs['attention_mask'].to(device)
new_labels_tensor = torch.tensor(new_labels, dtype=torch.long).to(device)

new_data = torch.utils.data.TensorDataset(new_input_ids, new_attention_mask, new_labels_tensor)
new_dataloader = DataLoader(new_data, batch_size=4)

# Set the model to evaluation mode
model.eval()

# Perform inference on the new dataset
predictions = []
with torch.no_grad():
    for batch in new_dataloader:
        input_ids, attention_mask, labels = batch

        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        probabilities = F.softmax(logits, dim=1)
        predicted_labels = torch.argmax(probabilities, dim=1)

        predictions.extend(predicted_labels.cpu().tolist())

# Map the predicted labels to sentiment classes
sentiment_classes = ['negative', 'positive']
predicted_sentiments = [sentiment_classes[label] for label in predictions]

# Add predicted sentiments to the DataFrame
new_df['predicted_sentiment'] = predicted_sentiments

# Print the new DataFrame with predicted sentiments
print(new_df)
from sklearn.metrics import accuracy_score

# Calculate accuracy score
true_labels = new_df['label'].values
predicted_labels = predictions

accuracy = accuracy_score(true_labels, predicted_labels)
print(f"Accuracy: {accuracy}")

                                                  text sentiment  label  \
1    The food was tasty here. We tried idly, ghee p...  positive      1   
2    A highly hyped place in the social media arena...  positive      1   
3    Excellent breakfast spot. Need to be there ear...  positive      1   
4    Awesome food. Go to the beach for sunrise and ...  positive      1   
5    Few places are this good. I come here without ...  positive      1   
..                                                 ...       ...    ...   
789                      Excellent pongal vada tasty …  positive      1   
791  Mylapore Kacheri Road Market Beautiful homemad...  positive      1   
821                                              Uhtso  positive      1   
835                                        Great tease  positive      1   
885                                     V. nice bond …  positive      1   

    predicted_sentiment  
1              positive  
2              positive  
3              positi

In [32]:
df1=pd.read_csv('/content/bread and choclate.csv')
df1 = df1[['hCCjke src', 'hCCjke src 2', 'hCCjke src 3','hCCjke src 4','wiI7pd']]
df1 = df1.rename(columns=new_column_names)
df1=df1.dropna()
df1 = df1.replace({
    'https://maps.gstatic.com/consumer/images/icons/2x/ic_star_rate_empty_14.png': 0,
    'https://maps.gstatic.com/consumer/images/icons/2x/ic_star_rate_14.png': 1
})
df1['star rating'] = df1[['1 star', '2 star', '3 star', '4 star']].sum(axis=1)
df1=df1[['text','star rating']]
df1 = df1[df1['star rating'] != 3]
df1['sentiment'] = df1['star rating'].apply(lambda x: 'negative' if x in [1, 2] else 'positive')
df1=df1[['text','sentiment']]
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader
from torch.nn import functional as F

# Load the fine-tuned BERT model and tokenizer
model_dir = "fine_tuned_bert_model"
model = BertForSequenceClassification.from_pretrained(model_dir)
tokenizer = BertTokenizer.from_pretrained(model_dir)

# Set device (CPU or GPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Example DataFrame with the new dataset
new_df = df1

# Map sentiment labels to integers
label_map = {'positive': 1, 'negative': 0}
new_df['label'] = new_df['sentiment'].map(label_map)

# Preprocess the new dataset
new_texts = new_df['text'].values
new_labels = new_df['label'].values

# Rest of the code for preprocessing and inference remains the same

# Preprocess the new dataset


new_inputs = tokenizer(new_texts.tolist(), padding=True, truncation=True, return_tensors='pt')
new_input_ids = new_inputs['input_ids'].to(device)
new_attention_mask = new_inputs['attention_mask'].to(device)
new_labels_tensor = torch.tensor(new_labels, dtype=torch.long).to(device)

new_data = torch.utils.data.TensorDataset(new_input_ids, new_attention_mask, new_labels_tensor)
new_dataloader = DataLoader(new_data, batch_size=4)

# Set the model to evaluation mode
model.eval()

# Perform inference on the new dataset
predictions = []
with torch.no_grad():
    for batch in new_dataloader:
        input_ids, attention_mask, labels = batch

        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        probabilities = F.softmax(logits, dim=1)
        predicted_labels = torch.argmax(probabilities, dim=1)

        predictions.extend(predicted_labels.cpu().tolist())

# Map the predicted labels to sentiment classes
sentiment_classes = ['negative', 'positive']
predicted_sentiments = [sentiment_classes[label] for label in predictions]

# Add predicted sentiments to the DataFrame
new_df['predicted_sentiment'] = predicted_sentiments

# Print the new DataFrame with predicted sentiments
print(new_df)
from sklearn.metrics import accuracy_score

# Calculate accuracy score
true_labels = new_df['label'].values
predicted_labels = predictions

accuracy = accuracy_score(true_labels, predicted_labels)
print(f"Accuracy: {accuracy}")

                                                  text sentiment  label  \
0    I love this place more and more everytime I vi...  positive      1   
3    I strongly recommend this place!\nThe food is ...  positive      1   
4    The best breakfast hub in town.\nGood food and...  positive      1   
5    Great cappuccino! Also tried their brownie cro...  positive      1   
6    Spacious cafe with a good ambience..!\nWe orde...  positive      1   
..                                                 ...       ...    ...   
280                   Loved the food and ambience here  positive      1   
281  Absolutely loved the place..great food lovely ...  positive      1   
282  Ordered croissants this morning through Dunzo,...  negative      0   
283  I had the bannana and fig smoothie and the alm...  positive      1   
284  Fantastic ambience and great food with amazing...  positive      1   

    predicted_sentiment  
0              positive  
3              positive  
4              positi